In [ ]:
import os
import pyvista as pv

pv.set_jupyter_backend('static')
os.environ['DISPLAY'] = ':99.0'
os.environ['PYVISTA_OFF_SCREEM'] = 'true'

# Settings

In [ ]:
mesh_path = '/mnt/d/knpob/4-data/20211229-DynaBreast4D/3dmd/6kmh_27marker_2/meshes'
export_folder = "output"

# Data loading

In [ ]:
import os

files = os.listdir(mesh_path)
files = [os.path.join(mesh_path, f.replace('.obj', '')) for f in files if '.obj' in f]
files.sort()

In [ ]:
import pyvista as pv
from mesh4d.analyse import crave

mesh = pv.read(files[0] + '.obj')
# mesh = crave.fix_pvmesh_disconnect(mesh)
texture = pv.read_texture(files[0] + '.jpg')

In [ ]:
mesh.plot(texture=texture)

In [ ]:
import numpy as np

def parse_vertex_color(mesh, texture) -> tuple:
    texture_img = np.array(texture.to_array())
    u, v = mesh.active_t_coords.T
    v_mapped = np.round((1 - v) * (texture_img.shape[0] - 1)).astype(int)
    u_mapped = np.round(u * (texture_img.shape[1] - 1)).astype(int)
    
    color_ls = texture_img[v_mapped, u_mapped]
    vertex_ls = np.array(mesh.points)
        
    return color_ls, vertex_ls

In [ ]:
color_ls, vertex_ls = parse_vertex_color(mesh, texture)
color_ls, vertex_ls

In [ ]:
from scipy.spatial import KDTree

tree = KDTree(vertex_ls)

# Marker extraction

## Gray scale threshold

In [ ]:
ds, idx = tree.query(vertex_ls, 10)
respond = np.sum(color_ls * [0.299, 0.587, 0.114], axis=1)
gray_ls = respond

In [ ]:
import matplotlib.pyplot as plt

plt.hist(respond, bins=100)
f"std {np.std(respond)} mean {np.mean(respond)} min {np.min(respond)} max {np.max(respond)}"

In [ ]:
mesh.plot(scalars=-respond, cmap='cool')

In [ ]:
import pyvista as pv

scene = pv.Plotter()
scene.add_points(vertex_ls[gray_ls < 100], point_size=2, color='goldenrod', render_points_as_spheres=True)
scene.add_mesh(mesh, scalars=gray_ls, cmap='gray')
scene.camera_position = 'xy'
scene.show()

## Local gray scale difference

In [ ]:
ds, idx = tree.query(vertex_ls, 10)
respond = np.max(np.abs(gray_ls[idx]  - np.expand_dims(gray_ls, axis=-1)), axis=1)

In [ ]:
plt.hist(respond, bins=100)
f"std {np.std(respond)} mean {np.mean(respond)} min {np.min(respond)} max {np.max(respond)}"

In [ ]:
mesh.plot(scalars=respond, cmap='cool')

In [ ]:
import pyvista as pv

scene = pv.Plotter()
scene.add_points(vertex_ls[respond > mean + 2 * std], point_size=2, color='goldenrod', render_points_as_spheres=True)
scene.camera_position = 'xy'
scene.show()

## Local gray scale gradients

In [ ]:
ds, idx = tree.query(vertex_ls, 100)
respond = np.max(np.abs(gray_ls[idx] - np.expand_dims(gray_ls, -1)) / (ds + 1e-5), axis=1)

In [ ]:
plt.hist(respond, bins=100)
f"std {np.std(respond)} mean {np.mean(respond)} min {np.min(respond)} max {np.max(respond)}"

In [ ]:
mesh.plot(scalars=respond, cmap='cool')

In [ ]:
import pyvista as pv

scene = pv.Plotter()
scene.add_points(vertex_ls[respond > mean + 2 * std], point_size=2, color='goldenrod', render_points_as_spheres=True)
scene.add_mesh(mesh, scalars=gray_ls, cmap='gray')
scene.camera_position = 'xy'
scene.show()

# Batch extraction

In [ ]:
stride = 10
total = len(files[::stride])

In [ ]:
def extract(mesh, texture, pnum=100, range=2):
    color_ls, vertex_ls = parse_vertex_color(mesh, texture)
    gray_ls = np.sum(color_ls * [0.299, 0.587, 0.114], axis=1)
    
    tree = KDTree(vertex_ls)
    ds, idx = tree.query(vertex_ls, pnum)
    
    respond = np.max(np.abs(gray_ls[idx] - np.expand_dims(gray_ls, -1)) / (ds + 1e-5), axis=1)
    mean = np.mean(respond)
    std = np.std(respond)
    marker_vertex = vertex_ls[respond > mean + range * std]
    # need to address empty issue

    return gray_ls, marker_vertex

In [ ]:
import os
from mesh4d import utils

scene = pv.Plotter()
scene.open_movie(os.path.join(export_folder, 'marker_3dmd.mp4'), framerate=round(120/stride))
marker = []

for idx, path in enumerate(files[::stride]):
    # load data
    mesh = pv.read(path + '.obj')
    texture = pv.read_texture(path + '.jpg')
    gray_ls, marker_vertex = extract(mesh, texture)

    # append extracted marker vertices
    marker.append(marker_vertex)

    # write visual
    scene.clear()
    scene.add_mesh(mesh, scalars=gray_ls, cmap='gray')

    if len(marker_vertex) > 0:
        scene.add_points(marker_vertex, point_size=2, color='goldenrod', render_points_as_spheres=True)

    scene.camera_position = 'xy'
    scene.write_frame()

    # print progress
    percent = (idx + 1) / total
    utils.progress_bar(percent, back_str=" processed the {}-th frame".format(idx))

scene.close()

In [ ]:
plt.bar(range(len(marker)), [len(frame) for frame in marker])

In [ ]:
utils.save_pkl_object(marker, export_folder=export_folder, export_name='marker_3dmd')

In [ ]:
scene = pv.Plotter()
scene.open_movie(os.path.join(export_folder, 'marker_vertex.mp4'), framerate=round(120/stride))

for idx, marker_vertex in enumerate(marker):
    scene.clear()

    if len(marker_vertex) > 0:
        scene.add_points(marker_vertex, point_size=2, color='goldenrod', render_points_as_spheres=True)

    scene.camera_position = 'xy'
    scene.write_frame()

    # print progress
    percent = (idx + 1) / total
    utils.progress_bar(percent, back_str=" processed the {}-th frame".format(idx))

scene.close()